In [83]:
import sklearn
import pickle
import pandas as pd
import sklearn.linear_model

import ML.features.features_with_sentence_list as features
from ML.natasha_func import ( get_doc, get_sents )
from tqdm import tqdm

In [75]:
dataset = pickle.load(open("dataset_1.pkl", "rb"))

In [80]:
authors = list(set(dataset['authors']))
dict_authors = dict()
reverse_dict_authors = dict()
for index in range(len(authors)):
    dict_authors[authors[index]] = index
    reverse_dict_authors[index] = authors[index]

target = []
for author in dataset["authors"]:
    target += [dict_authors[author]]

In [86]:
feature1 = []
feature2 = []
for book in tqdm(dataset['books']):
    doc = get_doc(book)
    sentences = get_sents(doc)

    feature1.append(features.avg_length_of_sentence_by_letters(sentences))
    feature2.append(features.capitalized_words_count_without_start_of_sentences(sentences))

100%|██████████| 100/100 [06:02<00:00,  3.62s/it]


In [95]:
import json
import compress_pickle
import os


DATA_PATH = "C:\\Users\\User\\PycharmProjects\\texts_attribution\\Данные"
observations = []
with open(os.path.join(DATA_PATH, "docs_info.json"), "r") as f_docs_info:
    docs_info = json.load(f_docs_info)
    authors = docs_info["authors"]
    file_names = docs_info["file_name"]

    for i, file_name in enumerate(tqdm(file_names)):
        author = authors[i]
        doc = compress_pickle.load(os.path.join(DATA_PATH, "Обработанные", file_name))
        sentences = get_sents(doc)
        observation = [features.capitalized_words_count_without_start_of_sentences(sentences),
                       features.avg_length_of_sentence_by_letters(sentences),
                       features.most_common_first_letter_in_sentences(sentences),
                       author]
        observations.append(observation)

df = pd.DataFrame(observations, columns=["count_caps_in_words", "avg_len_sent", "mc_first_let", "targets"])

100%|██████████| 516/516 [02:53<00:00,  2.98it/s]


In [107]:
df.rename(columns={"feature": "targets"}, inplace=True)

In [88]:
authors_seen = []
test_ixs = []
train_ixs = []
for ixs in range(len(df)):
    aut = df['targets'][ixs]
    if authors_seen.count(aut) < 2:
        authors_seen += [aut]
        test_ixs += [ixs]
    else: train_ixs += [ixs]

y_test = df.iloc[test_ixs]['targets']
x_test = df.iloc[test_ixs].drop(columns=["targets"])
y_train = df.iloc[train_ixs]['targets']
x_train = df.iloc[train_ixs].drop(columns=["targets"])

In [109]:
def validation(df, model):
    accuracy = []
    for shift in range(0, 10, 2):
        authors_seen = []
        test_ixs = []
        for ixs in range(len(df)):
            aut = df['targets'][ixs]
            if authors_seen.count(aut) < 2:
                authors_seen += [aut]
                test_ixs += [ixs + shift]
        train_ixs = list(set(range(len(df))) - set(test_ixs))
        y_test = df.iloc[test_ixs]['targets']
        x_test = df.iloc[test_ixs].drop(columns=["targets"])
        y_train = df.iloc[train_ixs]['targets']
        x_train = df.iloc[train_ixs].drop(columns=["targets"])

        rfc = model
        rfc.fit(x_train, y_train)
        accuracy.append(accuracy_score(y_test, rfc.predict(x_test)))
    return pd.Series(accuracy)

In [110]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [116]:
df = pd.read_csv("C:\\Users\\User\\PycharmProjects\\texts_attribution\\Данные\\first_dataset.csv")

In [122]:
df.rename(columns={func_name: func_name.split()[1] for func_name in df.columns[:-1]}, inplace=True)

In [126]:
df.rename(columns={"target": "targets"}, inplace=True)

In [128]:
logreg = LogisticRegression()
etc = ExtraTreesClassifier()


In [134]:
df.drop(columns="freq_word_from_adverbs", inplace=True)

In [ ]:
df.value_counts()

In [146]:
df.to_csv("first_dataset.csv", index=False)

In [140]:
import plotly.express as px

fig = px.imshow(df.drop(columns=["targets"]).corr(), text_auto=True)
fig.update_layout(height=1000)
fig.show()

In [133]:
df.columns.value_counts()

freq_word_from_adverbs                                2
avg_syllable_per_noun                                 1
freq_of_question_mark                                 1
freq_of_comma                                         1
freq_of_dot                                           1
avg_syllable_per_adverb                               1
avg_syllable_per_adjective                            1
avg_syllable_per_verb                                 1
capitalized_words_count_without_start_of_sentences    1
most_common_first_letter_in_sentences                 1
freq_word_from_verbs                                  1
freq_word_from_noun                                   1
average_lenght_of_verbs                               1
average_lenght_of_adverbs                             1
average_lenght_of_adjectives                          1
average_lenght_of_nouns                               1
freq_of_exclamation_mark                              1
freq_of_bracket                                 

In [ ]:
validation(df, )

In [79]:
/logreg.fit()

ValueError: Expected 2D array, got 1D array instead:
array=[ 67.49211356  83.09268293 135.44338967].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [74]:
logreg.predict()

array([1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0], dtype=int64)